# Loading LOB Data into Dataframe

In [2]:
import os
import pandas as pd
import re

## Iterate over files in data folder to form a DataFrame

In [3]:
data = []

# Define function to process each line the file
def parse_line(line):
    # trim and split
    parts = line.strip()[1:-1].split(', ', 2)
    # extracting values
    timestamp = float(parts[0])
    exchange = parts[1].strip('\'')
    # extracting bid and ask values
    bid_ask_data = parts[2][1:-1]
    bid_data = re.findall(r'\[\'bid\', \[\[(.*?)\]\]\]', bid_ask_data)
    bid = [list(map(int, pair.split(', '))) for pair in bid_data[0].split('], [')] if bid_data else []
    ask_data = re.findall(r'\[\'ask\', \[\[(.*?)\]\]\]', bid_ask_data)
    ask = [list(map(int, pair.split(', '))) for pair in ask_data[0].split('], [')] if ask_data else []
    return timestamp, exchange, bid, ask

directory = 'data/lob/'

# Loop through files in folder
for filename in os.listdir(directory):
    # Extract data from .txt files
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r') as file:
            for line in file:
                timestamp, exchange, bid, ask = parse_line(line)
                # add  data to list
                data.append({'Timestamp': timestamp, 'Exchange': exchange, 'Bid': bid, 'Ask': ask, 'File': filename}) 

# Create dataframe from list
df = pd.DataFrame(data)
df 

,Timestamp,Exchange,Bid,Ask,File
0,0.000,Exch0,[],[],UoB_Set01_2025-01-02LOBs.txt
1,0.279,Exch0,"[[1, 6]]",[],UoB_Set01_2025-01-02LOBs.txt
2,1.333,Exch0,"[[1, 6]]","[[800, 1]]",UoB_Set01_2025-01-02LOBs.txt
3,1.581,Exch0,"[[1, 6]]","[[799, 1]]",UoB_Set01_2025-01-02LOBs.txt
4,1.643,Exch0,"[[1, 6]]","[[798, 1]]",UoB_Set01_2025-01-02LOBs.txt
...,...,...,...,...,...
1037929,30599.418,Exch0,"[[323, 2], [104, 3], [63, 1], [44, 6]]","[[338, 1], [343, 2], [507, 4], [659, 1], [749,...",UoB_Set01_2025-01-06LOBs.txt
1037930,30599.449,Exch0,"[[323, 2], [99, 3], [63, 1], [44, 6]]","[[338, 1], [343, 2], [507, 4], [659, 1], [749,...",UoB_Set01_2025-01-06LOBs.txt
1037931,30599.635,Exch0,"[[323, 2], [99, 3], [63, 1], [44, 6]]","[[338, 1], [341, 2], [507, 4], [659, 1], [749,...",UoB_Set01_2025-01-06LOBs.txt
1037932,30599.697,Exch0,"[[323, 2], [249, 1], [99, 3], [44, 6]]","[[338, 1], [341, 2], [507, 4], [659, 1], [749,...",UoB_Set01_2025-01-06LOBs.txt


In [12]:
import os
import pandas as pd
import re

# Define function to process each line in the file
def parse_line(line):
    # trim and split
    parts = line.strip()[1:-1].split(', ', 2)
    # extracting values
    timestamp = float(parts[0])
    exchange = parts[1].strip('\'')
    # extracting bid and ask values
    bid_ask_data = parts[2][1:-1]
    bid_data = re.findall(r'\[\'bid\', \[\[(.*?)\]\]\]', bid_ask_data)
    bid = [list(map(int, pair.split(', '))) for pair in bid_data[0].split('], [')] if bid_data else []
    ask_data = re.findall(r'\[\'ask\', \[\[(.*?)\]\]\]', bid_ask_data)
    ask = [list(map(int, pair.split(', '))) for pair in ask_data[0].split('], [')] if ask_data else []
    return timestamp, exchange, bid, ask

# Define a function to extract first value from an array
def extract_first_value(arr):
    if len(arr) > 0:  # Check if the array is not empty
        return arr[0][0]  # Return the first value of the first array
    else:
        return None

directory = 'data/lob/'
output_csv = 'data/lob_output_data.csv'  # Output CSV file

# Loop through files in folder
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        data = []  # Initialize data list for each file
        with open(os.path.join(directory, filename), 'r') as file:
            # Loop through lines in text file
            for line in file:
                # Extract data from line
                timestamp, exchange, bid, ask = parse_line(line)
                # Extract date from filename
                date_match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)
                if date_match:
                    date = date_match.group(1)
                else:
                    date = None
                data.append({'Timestamp': timestamp, 'Exchange': exchange, 'Bid': bid, 'Ask': ask, 'Date': date})

        # Create DataFrame from list
        df = pd.DataFrame(data)
        df['Mid_Price'] = (df['Bid'].apply(lambda x: extract_first_value(x)) + df['Ask'].apply(lambda x: extract_first_value(x))) / 2
        
        # Append DataFrame to CSV file
        if os.path.exists(output_csv):
            df.to_csv(output_csv, mode='a', header=False, index=False)
        else:
            df.to_csv(output_csv, index=False)

In [13]:
df = pd.read_csv("data/lob_output_data.csv")

In [14]:
df

,Timestamp,Exchange,Bid,Ask,Date,Mid_Price
0,0.000,Exch0,[],[],2025-01-02,NaN
1,0.279,Exch0,"[[1, 6]]",[],2025-01-02,NaN
2,1.333,Exch0,"[[1, 6]]","[[800, 1]]",2025-01-02,400.5
3,1.581,Exch0,"[[1, 6]]","[[799, 1]]",2025-01-02,400.0
4,1.643,Exch0,"[[1, 6]]","[[798, 1]]",2025-01-02,399.5
...,...,...,...,...,...,...
1037929,30599.418,Exch0,"[[323, 2], [104, 3], [63, 1], [44, 6]]","[[338, 1], [343, 2], [507, 4], [659, 1], [749,...",2025-01-06,330.5
1037930,30599.449,Exch0,"[[323, 2], [99, 3], [63, 1], [44, 6]]","[[338, 1], [343, 2], [507, 4], [659, 1], [749,...",2025-01-06,330.5
1037931,30599.635,Exch0,"[[323, 2], [99, 3], [63, 1], [44, 6]]","[[338, 1], [341, 2], [507, 4], [659, 1], [749,...",2025-01-06,330.5
1037932,30599.697,Exch0,"[[323, 2], [249, 1], [99, 3], [44, 6]]","[[338, 1], [341, 2], [507, 4], [659, 1], [749,...",2025-01-06,330.5


## Seperate Bid and Ask Columns into Rows

In [ ]:
# Columns to melt
columns_to_melt = ['Bid', 'Ask']

# Melt the specified columns
melted_df = df.melt(id_vars=['Timestamp', 'Exchange', 'File'], value_vars=columns_to_melt,
                    var_name='Order Type', value_name='Value')

,Timestamp,Exchange,File,Order Type,Value
0,0.000,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,[]
1,0.279,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[[1, 6]]"
2,1.333,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[[1, 6]]"
3,1.581,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[[1, 6]]"
4,1.643,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[[1, 6]]"
...,...,...,...,...,...
2075863,30599.418,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[[338, 1], [343, 2], [507, 4], [659, 1], [749,..."
2075864,30599.449,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[[338, 1], [343, 2], [507, 4], [659, 1], [749,..."
2075865,30599.635,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[[338, 1], [341, 2], [507, 4], [659, 1], [749,..."
2075866,30599.697,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[[338, 1], [341, 2], [507, 4], [659, 1], [749,..."


## Reshaping the dataframe to show one bid and ask per row

In [ ]:
df = melted_df.explode('Value')

,Timestamp,Exchange,File,Order Type,Value
0,0.000,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,NaN
1,0.279,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[1, 6]"
2,1.333,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[1, 6]"
3,1.581,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[1, 6]"
4,1.643,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,"[1, 6]"
...,...,...,...,...,...
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[338, 1]"
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[341, 2]"
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[507, 4]"
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,"[659, 1]"


## Separate Price and Quantity from Values column

In [ ]:
df[['Price', 'Quantity']] = pd.DataFrame(df['Value'].apply(lambda x: x if isinstance(x, list) else []).tolist(), index=df.index)
df.drop(columns=['Value'], inplace=True)

,Timestamp,Exchange,File,Order Type,Price,Quantity
0,0.000,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,NaN,NaN
1,0.279,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,1.0,6.0
2,1.333,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,1.0,6.0
3,1.581,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,1.0,6.0
4,1.643,Exch0,UoB_Set01_2025-01-02LOBs.txt,Bid,1.0,6.0
...,...,...,...,...,...,...
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,338.0,1.0
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,341.0,2.0
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,507.0,4.0
2075867,30599.728,Exch0,UoB_Set01_2025-01-06LOBs.txt,Ask,659.0,1.0


## Extract Date from file column and store it as a Datetime in a Date coloumn

In [ ]:
# Extract date from 'File' column and add it as a new column
df['Date'] = df['File'].str.extract(r'(\d{4}-\d{2}-\d{2})')
# Convert the 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])
df.drop(columns=['File'], inplace=True)
df

,Timestamp,Exchange,Order Type,Price,Quantity,Date
0,0.000,Exch0,Bid,NaN,NaN,2025-01-02
1,0.279,Exch0,Bid,1.0,6.0,2025-01-02
2,1.333,Exch0,Bid,1.0,6.0,2025-01-02
3,1.581,Exch0,Bid,1.0,6.0,2025-01-02
4,1.643,Exch0,Bid,1.0,6.0,2025-01-02
...,...,...,...,...,...,...
2075867,30599.728,Exch0,Ask,338.0,1.0,2025-01-06
2075867,30599.728,Exch0,Ask,341.0,2.0,2025-01-06
2075867,30599.728,Exch0,Ask,507.0,4.0,2025-01-06
2075867,30599.728,Exch0,Ask,659.0,1.0,2025-01-06
